In [1]:
from gurobipy import *
import numpy as np
import time
from collections import defaultdict
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
from scipy.stats import norm
import math

In [5]:
capacidad = 200
precio_boleto = 475
media_noshow = 30
std_noshow = 15
compensacion = 800

c_u = 475
c_o = 800
F_Y = c_u/(c_u + c_o)
Y = norm.ppf(F_Y, loc=media_noshow, scale=std_noshow)
Y = math.ceil(Y)

print(f'Se deberían aceptar {Y} reservas adicionales, teniendo una nueva capacidad total de {capacidad + Y} boletos')

Se deberían aceptar 26 reservas adicionales, teniendo una nueva capacidad total de 226 boletos


In [7]:
Y = 20
Y_comp = Y - 1 # si 19 personas o menos no se presentan, tendré gente que no podrá subirse al avión
z = (Y-media_noshow)/std_noshow
F_z = norm.cdf(z)
print(f'La probabilidad de que se tenga que compensar pasajeros es de {F_z*100}%')

La probabilidad de que se tenga que compensar pasajeros es de 25.24925375469229%
